# Prepare Compass inputs, random groups within cell states

Due to large numbers of micropool clusters being contaminated with cells from other annotated groups, we will divide up cells within each celltype into groups of 20 (CM) and 50 (all cells)

## Setup

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import rc_context
import bbknn
import re
import sctk

/home/jovyan/my-conda-envs/scanpy/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sc.settings.verbosity = 0            # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white', color_map = 'Reds', figsize = (5, 5),dpi_save=300)

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.22.4 scipy==1.8.1 pandas==1.4.3 scikit-learn==1.1.1 statsmodels==0.13.2 python-igraph==0.9.11 pynndescent==0.5.7


In [3]:
# Perform pseudobulk on sc data

import scipy.sparse as sp

def pseudo_bulk(
        adata, groupby, use_rep='X', highly_variable=False, FUN=np.mean):
    """Make pseudo bulk data from grouped sc data
    """
    if adata.obs[groupby].dtype.name == 'category':
        group_attr = adata.obs[groupby].values
        groups = adata.obs[groupby].cat.categories.values
    else:
        group_attr = adata.obs[groupby].astype(str).values
        groups = np.unique(group_attr)
    n_level = len(groups)
    if highly_variable:
        if isinstance(highly_variable, (list, tuple)):
            if use_rep == 'raw':
                k_hv = adata.raw.var_names.isin(highly_variable)
            else:
                k_hv = adata.var_names.isin(highly_variable)
        else:
            k_hv = adata.var['highly_variable'].values
    if use_rep == 'X':
        x = adata.X
        features = adata.var_names.values
        if highly_variable:
            x = x[:, k_hv]
            features = features[k_hv]
    elif use_rep == 'raw':
        x = adata.raw.X
        features = adata.raw.var_names.values
        if highly_variable:
            x = x[:, k_hv]
            features = features[k_hv]
    elif use_rep in adata.layers.keys():
        x = adata.layers[use_rep]
        features = adata.var_names.values
        if highly_variable:
            x = x[:, k_hv]
            features = features[k_hv]
    elif use_rep in adata.obsm.keys():
        x = adata.obsm[use_rep]
        features = np.arange(x.shape[1])
    elif (isinstance(use_rep, np.ndarray) and
            use_rep.shape[0] == adata.shape[0]):
        x = use_rep
        features = np.arange(x.shape[1])
    else:
        raise KeyError(f'{use_rep} invalid.')
    summarised = np.zeros((n_level, x.shape[1]))
    for i, grp in enumerate(groups):
        k_grp = group_attr == grp
        if sp.issparse(x):
            summarised[i] = FUN(x[k_grp, :], axis=0)
        else:
            summarised[i] = FUN(x[k_grp, :], axis=0, keepdims=True)
    return pd.DataFrame(summarised.T, columns=groups, index=features)

def random_partition(
    adata,
    partition_size,
    groupby=None,
    method="random_even",
    key_added="partition_labels",
    random_state=0,
):
    np.random.seed(random_state)
    if groupby:
        if groupby not in adata.obs.columns:
            raise KeyError(f"{groupby} is not a valid obs annotation.")
        groups = adata.obs[groupby].unique()
        label_df = adata.obs[[groupby]].astype(str).rename(columns={groupby: key_added})
        for grp in groups:
            k = adata.obs[groupby] == grp
            grp_size = sum(k)
            n_partition = max(np.round(grp_size / partition_size).astype(int), 1)
            if method == "random":
                part_idx = np.random.randint(low=0, high=n_partition, size=grp_size)
            elif method == "random_even":
                part_sizes = list(map(len, np.array_split(np.arange(grp_size), n_partition)))
                part_idx = np.repeat(np.arange(n_partition), part_sizes)
                np.random.shuffle(part_idx)
            else:
                raise NotImplementedError(method)
            label_df.loc[k, key_added] = [f"{grp},{i}" for i in part_idx]
        adata.obs[key_added] = label_df[key_added]
    else:
        n_partition = max(np.round(adata.n_obs / partition_size).astype(int), 1)
        if method == "random":
            part_idx = np.random.randint(low=0, high=n_partition, size=adata.n_obs)
        elif method == "random_even":
            part_sizes = list(map(len, np.array_split(np.arange(adata.n_obs), n_partition)))
            part_idx = np.repeat(np.arange(n_partition), part_sizes)
            np.random.shuffle(part_idx)
        else:
            raise NotImplementedError(method)
        adata.obs[key_added] = part_idx.astype(str)

## Import suspension data

In [4]:
adata = sc.read('/nfs/team205/heart/anndata_objects/Foetal/final_annotation/FetalHearts13Aug.h5ad')
adata

AnnData object with n_obs × n_vars = 176050 × 3311
    obs: 'latent_RT_efficiency', 'latent_cell_probability', 'latent_scale', 'sangerID', 'combinedID', 'donor', 'donor_type', 'region_finest', 'facility', 'cell_or_nuclei', 'modality', 'kit_10x', 'flushed', 'scrublet_score', 'scrublet_leiden', 'cluster_scrublet_score', 'doublet_pval', 'doublet_bh_pval', 'n_genes', 'n_counts', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'HB_score', 'batch_key', '_scvi_batch', '_scvi_labels', 'annotation_Semih', 'leiden_scVI', 'cell_type_pre', 'pth', 'ito', 'FACSgate', 'leiden_scVI_0.05', 'leiden_scVI_0.01', 'leiden_scVI_0.1', 'leiden_scVI_0.15', 'leiden_scVI_0.2', 'leiden_scVI_0.25', 'leiden_scVI_0.3', 'leiden_scVI_0.35', 'leiden_scVI_0.4', 'cell_type', 'coarse_grain', 'fine_grain', 'region', 'sex', 'age', 'cycling', 'S_score', 'G2M_scor

In [5]:
# Recover log-normed data for all genes to adata.X (allows Compass to calculate reactions across all genes, not just HVG-subsetted!)
print(adata.shape)
adata=adata.raw.to_adata()
print(adata.shape)

(176050, 3311)
(176050, 36601)


In [6]:
# Check var_names are gene symbols, and the data is log-normed
print(adata.var.index[:2])
print(adata.X.data[:5])

Index(['MIR1302-2HG', 'FAM138A'], dtype='object', name='gene_name_multiome-1')
[0.09802545 0.09802545 0.41538525 0.41538525 0.6009918 ]


# Prepare data for Compass
"The input gene expression matrix can be either a tab-delimited text file (tsv) or a matrix market format (mtx) containing gene expression estimates (CPM, TPM, or similar scaled units) with one row per gene, one column per sample."

## .mtx method (gene x sampleID, 3 files)
This allows you to specify the 'samples', so rather than calculating reactions in all single cells individually (or micro-pooled) we can make our own "samples" (pseudobulking them)

Need to ensure the samples are representative of the sample. Here we use a new batch key of cell state and the previous batch key  to form the 'samples'
This requires 3 files, to inputted thus ```--data expression.mtx genes.tsv sample_names.tsv```

Another option would be to use Milo or Metacell to make small samples, below I am going to just use the batch key to make samples within each cell state.

In [7]:
# make random groups of size 20 within each cell state
adata.obs['fine_grain'].value_counts()

GreatVesselAdventitialFibroblasts        18605
MyocardialInterstitialFibroblasts        15073
VentricularCardiomyocytesCompact         13740
GreatVesselProgenitors                   13175
VentricularCardiomyocytesPRRX1pos        11627
VentricularCardiomyocytesCycling         11297
MacrophagesLYVE1pos                       8864
GreatVesselSmoothMuscleCells              7870
EndocardialCells                          7418
ValveInterstitialCells                    7222
VentricularCardiomyocytesTrabeculated     6936
AtrialCardiomyocytesLeft                  6304
AtrialCardiomyocytesRight                 5452
Cardiofibromyocytes                       5278
CoronaryEndothelialCellsCapillary         4834
CoronaryVesselAdventitialFibroblasts      2683
CoronaryPericytes                         2077
SatelliteGlialCells                       2008
Myofibroblasts                            1826
MacrophagesCX3CR1pos                      1609
VentricularConductionSystemDistal         1568
InnateLymphoi

In [8]:
# Make random partition using sctk
random_partition(adata, partition_size=20, groupby="fine_grain", key_added="partition_20")

In [9]:
adata.obs

,latent_RT_efficiency,latent_cell_probability,latent_scale,sangerID,combinedID,donor,donor_type,region_finest,facility,cell_or_nuclei,...,region,sex,age,cycling,S_score,G2M_score,phase,stress_score,hb1_score,partition_20
barcode,,,,,,,,,,,,,,,,,,,,,
BHF_F_Hea10402917_TGGCTGGGTTGTGGAG-1,5.218614,0.999939,123665.210938,BHF_F_Hea10402917,na,C86,na,Heart,Sanger,Cell,...,WholeHeart,Male,12W5D,Yes,-0.005998,-0.126205,G1,0.494416,-0.223034,"CoronaryEndothelialCellsCapillary,58"
BHF_F_Hea10402917_ATGTGTGTCGAACGGA-1,3.109560,0.999991,10201.502930,BHF_F_Hea10402917,na,C86,na,Heart,Sanger,Cell,...,WholeHeart,Male,12W5D,Yes,0.656214,0.093990,S,1.074951,-0.143935,"InnateLymphoidCells,53"
BHF_F_Hea10402917_CAGGTGCAGGCTATCT-1,3.181446,0.999989,10217.305664,BHF_F_Hea10402917,na,C86,na,Heart,Sanger,Cell,...,WholeHeart,Male,12W5D,Yes,0.386814,1.163369,G2M,0.460513,-0.138523,"BCells,5"
BHF_F_Hea10402917_TACAGTGTCCAGTATG-1,3.101654,0.999993,9392.824219,BHF_F_Hea10402917,na,C86,na,Heart,Sanger,Cell,...,WholeHeart,Male,12W5D,Yes,0.640767,0.132809,S,0.994759,-0.053963,"InnateLymphoidCells,59"
BHF_F_Hea10402917_ACCAGTATCCACTCCA-1,3.109215,0.999989,9608.674805,BHF_F_Hea10402917,na,C86,na,Heart,Sanger,Cell,...,WholeHeart,Male,12W5D,No,-0.107722,-0.040233,G1,0.789726,-0.090271,"ProBCells,3"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BHF_F_Hea11933675_BHF_F_Hea11596628_GCTCATTGTCAGGCCA-1,0.756771,0.978465,3959.675781,BHF_F_Hea11933675,BHF_F_Hea11933675_BHF_F_Hea11596628,C98,na,Heart,Sanger,Nuclei,...,WholeHeart,Male,9W1D,No,-0.118187,-0.089172,G1,-0.454305,0.097133,"GreatVesselAdventitialFibroblasts,929"
BHF_F_Hea11933675_BHF_F_Hea11596628_AAGACATAGCATGACT-1,0.756599,0.978873,4021.003662,BHF_F_Hea11933675,BHF_F_Hea11933675_BHF_F_Hea11596628,C98,na,Heart,Sanger,Nuclei,...,WholeHeart,Male,9W1D,Yes,-0.060168,0.490866,G2M,0.222192,0.669477,"GreatVesselSmoothMuscleCells,7"
BHF_F_Hea11933675_BHF_F_Hea11596628_AGACAATAGGAGTAAT-1,0.744038,0.971963,3929.745605,BHF_F_Hea11933675,BHF_F_Hea11933675_BHF_F_Hea11596628,C98,na,Heart,Sanger,Nuclei,...,WholeHeart,Male,9W1D,No,-0.152945,-0.124303,G1,-0.028035,0.766656,"Monocytes,8"


In [10]:
# Pseudobulk use Ni's psudobulk function
pseudo_bulk = pseudo_bulk(adata,
                         groupby='partition_20')

In [11]:
print(pseudo_bulk.shape)
pseudo_bulk.head(3)

(36601, 8802)


,"AtrialCardiomyocytesCycling,0","AtrialCardiomyocytesCycling,1","AtrialCardiomyocytesCycling,10","AtrialCardiomyocytesCycling,11","AtrialCardiomyocytesCycling,12","AtrialCardiomyocytesCycling,13","AtrialCardiomyocytesCycling,14","AtrialCardiomyocytesCycling,15","AtrialCardiomyocytesCycling,2","AtrialCardiomyocytesCycling,3",...,"VentricularConductionSystemProximal,32","VentricularConductionSystemProximal,33","VentricularConductionSystemProximal,34","VentricularConductionSystemProximal,35","VentricularConductionSystemProximal,4","VentricularConductionSystemProximal,5","VentricularConductionSystemProximal,6","VentricularConductionSystemProximal,7","VentricularConductionSystemProximal,8","VentricularConductionSystemProximal,9"
MIR1302-2HG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM138A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# make sample_names.tsv
pseudo_bulk.columns.to_frame().to_csv('/nfs/team205/heart/compass/fetal_allcells_random20/sample_names.tsv', sep="\t", index=False, header=False)

# make genes.tsv
genes = pseudo_bulk.index.to_list()
genes = pd.DataFrame(genes)
genes.to_csv('/nfs/team205/heart/compass/fetal_allcells_random20/genes.tsv', sep="\t", index = False, header=False)

# make expression.mtx
from scipy.io import mmwrite, mmread
from scipy.sparse import csr_matrix

# Count Matrix (log normed, all genes, not HVG subsetted)
sparce_mtx = csr_matrix(pseudo_bulk)
sparce_mtx = sparce_mtx
mmwrite('/nfs/team205/heart/compass/fetal_allcells_random20/expression.mtx', sparce_mtx)

# Export cell metadata

In [13]:
# Alter so it includes things you need (a reference e.g. barcode and a cell type e.g. in fine_grain)
adata.obs.to_csv('/nfs/team205/heart/compass/fetal_allcells_random20/cell_metadata.csv')

# Run Compass

## bsub command

```
bsub \
-G teichlab \
-q basement \
-M300000 \
-R "select[mem>300000] rusage[mem=300000] span[hosts=1]" \
-n 30 \
-J all_pool50 \
-o 'log/compass.%J.out' \
-e 'log/compass.%J.err' \
'/software/singularity-v3.6.4/bin/singularity run -B /nfs,/lustre -B ~/COMPASSResources:/usr/local/lib/python3.8/site-packages/compass/Resources /nfs/cellgeni/singularity/images/compass-v0.9.10.2-cplex-v20.10.sif compass --data /nfs/team205/heart/compass/fetal_allcells/fetal_allcells_count_matrix.tsv --species homo_sapiens --precache --calc-metabolites --output-dir /nfs/team205/heart/compass/fetal_allcells/out/ --detailed-perf --num-processes 30 --microcluster-size 50'
```

remove --microcluster-size flag to run on single-cell data

if using .mtx methods then need to change --data input e.g. "--data expression.mtx genes.tsv sample_names.tsv"